# Denoising diffusion implicit models

This notebook is a follow-up of the [DDPM introduction](intro_1_ddpm.ipynb) and provides a brief discussion of **denoising diffusion implicit models** (DDIMs). They have been introduced in [[Song et al., 2021](https://openreview.net/forum?id=St1giarCHLP)] as a generalization of DDPMs. In particular, a non-Markovian noising process is constructed such that it features exactly the same marginals as in the strictly Markovian DDPM formulation. While this leads to the same training objective, it also allows for skipping steps in the generative process. This way the sampling procedure is accelerated.

## Noising process

In [[Song et al., 2021](https://openreview.net/forum?id=St1giarCHLP)] a family of non-Markovian noising processes $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{1:T} | \boldsymbol{x}_0)$, indexed by a vector $\boldsymbol{\sigma} = (\sigma_1, \ldots, \sigma_T)$ with $\sigma_t \geq 0$ for all $t = 1, \ldots, T$, has been proposed. No matter the exact values of those hyperparameters, the process has the same marginal $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_t | \boldsymbol{x}_0) = \mathcal{N}(\boldsymbol{x}_t | \sqrt{\bar{\alpha}_t} \boldsymbol{x}_0, (1 - \bar{\alpha}_t) \boldsymbol{I})$ as in the strictly Markovian DDPM formulation. This construction is based on the ("reversely" ordered) factorization
$$
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{1:T} | \boldsymbol{x}_0) =
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_T | \boldsymbol{x}_0)
\prod_{t=2}^T q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0).
$$
The specific terms are defined as follows:
$$
\begin{align*}
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_T | \boldsymbol{x}_0) &=
\mathcal{N} \left( \boldsymbol{x}_T | \sqrt{\bar{\alpha}_T} \boldsymbol{x}_0,
(1 - \bar{\alpha}_T) \boldsymbol{I} \right), \\
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0) &=
\mathcal{N} \left( \boldsymbol{x}_{t-1} | \sqrt{\bar{\alpha}_{t-1}} \boldsymbol{x}_0 +
\sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot
\frac{\boldsymbol{x}_t - \sqrt{\bar{\alpha}_t} \boldsymbol{x}_0}{\sqrt{1 - \bar{\alpha}_t}},
\sigma_t^2 \boldsymbol{I} \right), \quad \text{for } t = 2, \ldots, T.
\end{align*}
$$

It can be noted that $\sigma_t^2 \boldsymbol{I}$ is simply variance of $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0)$. It is noted that $q(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0) = \mathcal{N}(\boldsymbol{x}_{t-1} | \tilde{\boldsymbol{\mu}}_t (\boldsymbol{x}_t, \boldsymbol{x}_0), \tilde{\beta}_t \boldsymbol{I})$ with $\tilde{\beta}_t = \frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t} \beta_t$, the corresponding quantity of the DDPM, does not have such a degree of freedom. For $\sigma_t \rightarrow 0$ this distribution concentrates more and more at a single value, until it becomes fully deterministic.

An alternative way of writing the distribution of the noised steps $\boldsymbol{x}_{1:T}$ given an initial $\boldsymbol{x}_0$ is $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{1:T} | \boldsymbol{x}_0) = q_{\boldsymbol{\sigma}}(\boldsymbol{x}_1 | \boldsymbol{x}_0) \prod_{t=2}^T q_{\boldsymbol{\sigma}}(\boldsymbol{x}_t | \boldsymbol{x}_{t-1}, \boldsymbol{x}_0)$. Here, the "forward" step can be obtained through Bayes' theorem
$$
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_t | \boldsymbol{x}_{t-1}, \boldsymbol{x}_0) =
\frac{q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0)
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_t | \boldsymbol{x}_0)}
{q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_0)}.
$$

Note that the noising process has been specifically designed to have the same marginals as the Markovian formulation. The proof of this property is not of importance here.

## Generative process

As a next step, a trainable generative process $p_{\boldsymbol{\theta}}(\boldsymbol{x}_{0:T}) = p(\boldsymbol{x}_T) \prod_{t=1}^T p_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t)$ is constructed. Given a noisy $\boldsymbol{x}_t$ at a time step $t$, the initial state is represented as $\boldsymbol{x}_0 = \frac{1}{\sqrt{\bar{\alpha}_t}} (\boldsymbol{x}_t - \sqrt{1-\bar{\alpha}_t} \boldsymbol{\epsilon})$. Here, $\boldsymbol{\epsilon}$ is the corresponding noise vector. With the $\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t)$-based $\boldsymbol{x}_0$-estimate
$$
f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t) =
\frac{1}{\sqrt{\bar{\alpha}_t}} \left( \boldsymbol{x}_t - \sqrt{1 - \bar{\alpha}_t}
\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t) \right)
$$
the following definition of $p_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t)$ uses the construction of $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0)$ from above:
$$
p_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t) =
\begin{cases}
\mathcal{N}(\boldsymbol{x}_0 | f_{\boldsymbol{\theta}}^{(1)}(\boldsymbol{x}_1),
\sigma_1^2 \boldsymbol{I}) & \text{if } t = 1, \\
q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t)) & \text{otherwise}.
\end{cases}
$$

It is noted that $q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0)$ and $p_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t)$ share the same variance. In contrast, $q(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t, \boldsymbol{x}_0) = \mathcal{N}(\boldsymbol{x}_{t-1} | \tilde{\boldsymbol{\mu}}_t (\boldsymbol{x}_t, \boldsymbol{x}_0), \tilde{\beta}_t \boldsymbol{I})$ and $p_{\boldsymbol{\theta}}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t) = \mathcal{N}(\boldsymbol{x}_{t-1} | \boldsymbol{\mu}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t), \sigma_t^2 \boldsymbol{I})$ of the DDPM formulation do not have the same variance necessarily.

## Training objective

Analogous to the DDPM, the training objective can be written as $L_{\boldsymbol{\sigma}} = \mathbb{E}_{q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{0:T})}[\log(q_{\boldsymbol{\sigma}}(\boldsymbol{x}_{1:T} | \boldsymbol{x}_0) - p_{\boldsymbol{\theta}}(\boldsymbol{x}_{0:T}))]$. An interesting observation of the relationship to the simplified DDPM objective $L_{\text{simple}}$ is then found in the DDIM paper [[Song et al., 2021](https://openreview.net/forum?id=St1giarCHLP)]. It justifies the use of $L_{\text{simple}}$, which simply discarded some weighting factors, as a loss function. If the model $\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t)$ does not share parameters over different $t$, which means that $t$ is an index of the model rather than an input, both training objectives are perfectly equivalent. The reason is that the loss terms are independent from each other.

One can even further argue that if the noise model does share parameters over $t$, which means that $t$ is indeed an input, the objective $L_{\text{simple}}$ still corresponds to an appropriately reweighted version of $L_{\boldsymbol{\sigma}}$. For the sake of completeness, the loss is rewritten here as
$$
L_{\text{simple}} = \mathbb{E}_{\mathcal{U}(t|1, T), q(\boldsymbol{x}_0),
\mathcal{N}(\boldsymbol{\epsilon} | \boldsymbol{0}, \boldsymbol{I})}
\left[ \left\lVert \boldsymbol{\epsilon} - \boldsymbol{\epsilon}_{\boldsymbol{\theta}} \left(
\sqrt{\bar{\alpha}_t} \boldsymbol{x}_0 + \sqrt{1-\bar{\alpha}_t} \boldsymbol{\epsilon}, t \right)
\right\rVert^2 \right].
$$

## Accelerated sampling

After a model $\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t)$ has been successfully trained, based on $p_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_{t-1} | \boldsymbol{x}_t)$ one can recursively realize samples from the generative process as described below. First, with the estimate $\boldsymbol{x}_0 \approx f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t)$ one can write
$$
q_{\boldsymbol{\sigma}} \left( \boldsymbol{x}_{t-1} | \boldsymbol{x}_t,
f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t) \right) =
\mathcal{N} \left( \boldsymbol{x}_{t-1} | \sqrt{\bar{\alpha}_{t-1}} \cdot
f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t) +
\sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot
\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t),
\sigma_t^2 \boldsymbol{I} \right).
$$
Now one can easily see that, given a state $\boldsymbol{x}_t$, samples at the previous time step $\boldsymbol{x}_{t-1}$ can be simulated according to
$$
\boldsymbol{x}_{t-1} =
\sqrt{\bar{\alpha}_{t-1}} \cdot f_{\boldsymbol{\theta}}^{(t)}(\boldsymbol{x}_t) +
\sqrt{1 - \bar{\alpha}_{t-1} - \sigma_t^2} \cdot
\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_t, t) +
\sigma_t^2 \boldsymbol{\epsilon}.
$$

Different choices of the free hyperparameters $\boldsymbol{\sigma} = (\sigma_1, \ldots, \sigma_T)$ yield different models from the discussed family of noising and generative processes. One can easily verify that a certain choice of $\sigma_t$ corresponds to a DDPM with a perfectly Markovian forward process. The choice $\sigma_t = 0$ for all $t=1, \ldots T$ yields a DDPM. In this case, after an initial $\boldsymbol{x}_T \sim p(\boldsymbol{x}_T)$ has been drawn, the generative sampling process actually becomes deterministic.

A certain consistency property emerges for a largely deterministic sampling process. If the randomness comes entirely from the random initial noise state, one can smoothly interpolate in the DDIM latent (noise) space. This stands in contrast to DDPMs, where an identical noise state results in different generations for each run.

For a given pretrained model, one can accelerate the sampling procedure by skipping some of the steps in the generative process. This holds for any choice of $\sigma_t$, including the DDPM or DDIM as special cases. While all time steps $(1, \ldots, T)$ can be used in training, during data generation one can focus on a subsequence $\tau = (\tau_1, \ldots \tau_S)$ with a shorter length $S \leq T$ and $\tau_S = T$. The generative recursion can then be written as
$$
\boldsymbol{x}_{\tau_{i-1}} =
\sqrt{\bar{\alpha}_{\tau_{i-1}}} \cdot f_{\boldsymbol{\theta}}^{(\tau_i)}(\boldsymbol{x}_{\tau_i}) +
\sqrt{1 - \bar{\alpha}_{\tau_{i-1}} - \sigma_{\tau_i}^2} \cdot
\boldsymbol{\epsilon}_{\boldsymbol{\theta}}(\boldsymbol{x}_{\tau_i}, \tau_i) +
\sigma_{\tau_i}^2 \boldsymbol{\epsilon}.
$$